<a href="https://colab.research.google.com/github/danielhcg/NSF-CyberTraining/blob/main/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping
from keras.optimizers import SGD

from sklearn.feature_selection import SelectKBest, f_classif

In [3]:
# Load dataset
df = pd.read_csv('./02-16-2018.csv')  # CSE-CIC-IDS 2018 (02-16-2018)

<ipython-input-3-2b4c369fb987>:2: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./02-16-2018.csv')  # CSE-CIC-IDS 2018 (02-16-2018)


In [4]:
cols = df.columns
cols = cols.map(lambda x: x.replace(' ', '_'))
df.columns = cols
print(df.head())

  Dst_Port Protocol            Timestamp Flow_Duration Tot_Fwd_Pkts  \
0        0        0  16/02/2018 08:27:23     112640768            3   
1        0        0  16/02/2018 08:30:12     112641773            3   
2    35605        6  16/02/2018 08:26:55      20784143           23   
3        0        0  16/02/2018 08:33:01     112640836            3   
4       23        6  16/02/2018 08:27:59            20            1   

  Tot_Bwd_Pkts TotLen_Fwd_Pkts TotLen_Bwd_Pkts Fwd_Pkt_Len_Max  \
0            0               0               0               0   
1            0               0               0               0   
2           44            2416            1344             240   
3            0               0               0               0   
4            1               0               0               0   

  Fwd_Pkt_Len_Min  ... Fwd_Seg_Size_Min Active_Mean Active_Std Active_Max  \
0               0  ...                0           0        0.0          0   
1               0  ...

In [5]:
#replace infinit number
df=df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

#drop null
df.dropna(how='all')
print('Data type of each column of Dataframe :')
df.info(verbose=True)
df = df.sort_values('Timestamp')
df.drop(df.tail(1).index, inplace=True)
df = df.reset_index(drop=True)
df = df.drop(columns=['Timestamp', 'Dst_Port', 'Protocol'])
print(df['Label'].value_counts())

Data type of each column of Dataframe :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   Dst_Port           1048575 non-null  object
 1   Protocol           1048575 non-null  object
 2   Timestamp          1048575 non-null  object
 3   Flow_Duration      1048575 non-null  object
 4   Tot_Fwd_Pkts       1048575 non-null  object
 5   Tot_Bwd_Pkts       1048575 non-null  object
 6   TotLen_Fwd_Pkts    1048575 non-null  object
 7   TotLen_Bwd_Pkts    1048575 non-null  object
 8   Fwd_Pkt_Len_Max    1048575 non-null  object
 9   Fwd_Pkt_Len_Min    1048575 non-null  object
 10  Fwd_Pkt_Len_Mean   1048575 non-null  object
 11  Fwd_Pkt_Len_Std    1048575 non-null  object
 12  Bwd_Pkt_Len_Max    1048575 non-null  object
 13  Bwd_Pkt_Len_Min    1048575 non-null  object
 14  Bwd_Pkt_Len_Mean   1048575 non-null  object
 15  Bwd_Pkt_L

In [6]:
df.drop(df.loc[df['Label'] == 'DoS attacks-SlowHTTPTest'].index, inplace=True)

In [7]:
df.Label[df.Label == 'Benign'] = 0
df.Label[df.Label == 'DoS attacks-Hulk'] = 1
df["Label"] = df["Label"].astype('Int64')
print(df['Label'].value_counts())

Label
1    461912
0    446772
Name: count, dtype: Int64


In [8]:
#Prepare data for feature selection
train_df = df.copy() # Use .iloc to avoid a SettingWithCopyWarning
y = train_df.loc[0:52147,'Label'].astype('Int64')
X = train_df.drop(columns=['Label']).astype(np.float64)
X = X.loc[0:52147,:]

In [9]:
print(X.head(5))
print(y.value_counts())

   Flow_Duration  Tot_Fwd_Pkts  Tot_Bwd_Pkts  TotLen_Fwd_Pkts  \
0    112640723.0           3.0           0.0              0.0   
1      2146470.0          14.0          12.0           1335.0   
2    112640737.0           3.0           0.0              0.0   
3          721.0           1.0           1.0            300.0   
4    112640647.0           3.0           0.0              0.0   

   TotLen_Bwd_Pkts  Fwd_Pkt_Len_Max  Fwd_Pkt_Len_Min  Fwd_Pkt_Len_Mean  \
0              0.0              0.0              0.0          0.000000   
1           2273.0            744.0              0.0         95.357143   
2              0.0              0.0              0.0          0.000000   
3            329.0            300.0            300.0        300.000000   
4              0.0              0.0              0.0          0.000000   

   Fwd_Pkt_Len_Std  Bwd_Pkt_Len_Max  ...  Fwd_Act_Data_Pkts  Fwd_Seg_Size_Min  \
0         0.000000              0.0  ...                0.0               0.0   
1 

#Feature Selection

In [10]:
#Feature selection using SelectKBest
selector = SelectKBest(f_classif, k=20)  # Select top 20 features
X_new = selector.fit_transform(X, y)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [30 31 32 42 44 48 49 54 55 56 57 58 59] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [11]:
#Get the selected feature names
selected_features = X.columns[selector.get_support()]
print(f'Selected features: {selected_features}')

Selected features: Index(['Tot_Fwd_Pkts', 'TotLen_Fwd_Pkts', 'Fwd_Pkt_Len_Max',
       'Fwd_Pkt_Len_Mean', 'Fwd_Pkt_Len_Std', 'Flow_IAT_Std', 'Flow_IAT_Max',
       'Fwd_IAT_Tot', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_Header_Len',
       'Pkt_Len_Max', 'Pkt_Len_Std', 'Fwd_Seg_Size_Avg', 'Subflow_Fwd_Pkts',
       'Subflow_Fwd_Byts', 'Fwd_Act_Data_Pkts', 'Idle_Mean', 'Idle_Max',
       'Idle_Min'],
      dtype='object')


In [12]:
#Normalize selected features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_new)

#_

In [13]:
# Define sequence length and overlap
seq_length = 10  # Sequence length
seq_overlap = 5  # Overlap between sequences

In [14]:
def generate_sequences(X, y):
    X_seqs = []
    y_seqs = []
    for i in range(0, len(X) - seq_length + 1, seq_overlap):
        X_seqs.append(X.iloc[i:i+seq_length, :].to_numpy())
        y_seqs.append(y.iloc[i+seq_length+1])
    return np.array(X_seqs), np.array(y_seqs)

In [15]:
X_seqs, y_seqs = generate_sequences(pd.DataFrame(X_scaled), y)

In [16]:
# split dataset
l = len(X_seqs)
train_size = int(0.8 * l)
val_size = int(0.1 * l)
test_size = l - train_size - val_size
train_X = X_seqs[:train_size,:]
train_y = y_seqs[:train_size]
val_X = X_seqs[train_size:train_size+val_size,:]
val_y = y_seqs[train_size:train_size+val_size]
test_X = X_seqs[train_size+val_size:,:]
test_y = y_seqs[train_size+val_size:]

In [26]:
normal_indices = [i for i, x in enumerate(train_y) if x == 0]
abnormal_indices = [i for i, x in enumerate(train_y) if x == 1]

print('normal counts in training set: ', len(normal_indices))
print('abnormal counts in training set: ', len(abnormal_indices))
print('\n')
normal_indices = [i for i, x in enumerate(val_y) if x == 0]
abnormal_indices = [i for i, x in enumerate(val_y) if x == 1]

print('normal counts in validation set: ', len(normal_indices))
print('abnormal counts in validation set: ', len(abnormal_indices))
print('\n')
normal_indices = [i for i, x in enumerate(test_y) if x == 0]
abnormal_indices = [i for i, x in enumerate(test_y) if x == 1]

print('normal counts in test set: ', len(normal_indices))
print('abnormal counts in test set: ', len(abnormal_indices))


normal counts in training set:  2749
abnormal counts in training set:  2749


normal counts in validation set:  517
abnormal counts in validation set:  517


normal counts in test set:  515
abnormal counts in test set:  515


In [17]:
import random

def sample_even_distribution(X_seqs, y_label):
    normal_indices = [i for i, x in enumerate(y_label) if x == 0]
    abnormal_indices = [i for i, x in enumerate(y_label) if x == 1]

    min_samples = min(len(normal_indices), len(abnormal_indices))

    sampled_indices = []
    for i in range(min_samples):
        sampled_indices.append(normal_indices[i])
        sampled_indices.append(abnormal_indices[i])

    np.random.shuffle(sampled_indices)

    X = [X_seqs[i] for i in sampled_indices]
    y = [y_label[i] for i in sampled_indices]

    return np.array(X), np.array(y)

In [18]:
train_X, train_y = sample_even_distribution(train_X, train_y)
val_X, val_y = sample_even_distribution(val_X, val_y)
test_X, test_y = sample_even_distribution(test_X, test_y)

In [19]:
#Define model architecture with ReLU activation function
model = Sequential()

model.add(LSTM(units=64, input_shape=(seq_length, X_new.shape[1]), return_sequences=True))
model.add(LSTM(units=32, return_sequences=False))
model.add(Dropout(0.2))

model.add(Activation('relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [20]:
# Compile model with adam optimizer
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
print(train_X.shape, train_y.shape, val_X.shape, val_y.shape)
model.summary()

(5498, 10, 20) (5498,) (1034, 10, 20) (1034,)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            21760     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 activation (Activation)     (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 34209 (133.63 KB)
Trainable params: 34209 (133.63 KB)
Non-trainable params: 0 (0.00 Byte)
______________________________________

In [22]:
# Train model
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
history = model.fit(train_X, train_y, epochs=100, batch_size=16, validation_data=(val_X, val_y), callbacks=[early_stop])

Epoch 1/100
344/344 [==============================] - 12s 14ms/step - loss: 0.1659 - accuracy: 0.9576 - val_loss: 1.0369 - val_accuracy: 0.6335
Epoch 2/100
344/344 [==============================] - 4s 12ms/step - loss: 0.1291 - accuracy: 0.9651 - val_loss: 0.9248 - val_accuracy: 0.6228
Epoch 3/100
344/344 [==============================] - 7s 21ms/step - loss: 0.1240 - accuracy: 0.9640 - val_loss: 0.9046 - val_accuracy: 0.6209
Epoch 4/100
344/344 [==============================] - 7s 20ms/step - loss: 0.1213 - accuracy: 0.9645 - val_loss: 1.1992 - val_accuracy: 0.6151
Epoch 5/100
344/344 [==============================] - 4s 11ms/step - loss: 0.1185 - accuracy: 0.9654 - val_loss: 1.1669 - val_accuracy: 0.6054
Epoch 6/100
344/344 [==============================] - 5s 14ms/step - loss: 0.1159 - accuracy: 0.9654 - val_loss: 0.9494 - val_accuracy: 0.6257
Epoch 7/100
344/344 [==============================] - 5s 14ms/step - loss: 0.1161 - accuracy: 0.9649 - val_loss: 1.0684 - val_accuracy

In [23]:
# Evaluate model
predictions = model.predict(test_X)
binary_predictions = (predictions > 0.5).astype(int)

33/33 [==============================] - 1s 6ms/step


In [24]:
# Test model
from sklearn.metrics import accuracy_score
tp = tn = fp = fn = 0
predictn = predictions.flatten().round().tolist()
Y_testn = test_y.tolist()
for i in range(len(Y_testn)):
    if predictn[i] == 1 and Y_testn[i] == 1:
        tp += 1
    elif predictn[i] == 0 and Y_testn[i] == 0:
        tn += 1
    elif predictn[i] == 0 and Y_testn[i] == 1:
        fp += 1
    elif predictn[i] == 1 and Y_testn[i] == 0:
        fn += 1

In [25]:
tpr = tp / (tp + fn)
fnr = fn / (tp + fn)
tnr = tn / (tn + fp)
fpr = fp / (tn + fp)
print('tpr', tpr)
print('fnr', fnr)
print('tnr', tnr)
print('fpr', fpr)
scores = model.evaluate(test_X, test_y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy = accuracy_score(test_y, binary_predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

tpr 0.9958419958419958
fnr 0.004158004158004158
tnr 0.9344262295081968
fpr 0.06557377049180328
accuracy: 96.31%
Accuracy: 96.31%
